In [30]:
def combine_drawn_boxes(rectangles, proximity_threshold=30):
    """
    Combine bounding boxes that are close to each other.
    :param rectangles: List of drawn bounding boxes [(x, y, w, h), ...]
    :param proximity_threshold: Distance threshold to combine boxes
    :return: List of combined bounding boxes [(x, y, w, h), ...]
    """
    combined_boxes = []

    for box in rectangles:
        x, y, w, h = box
        merged = False
        for i, (cx, cy, cw, ch) in enumerate(combined_boxes):
            # Check if boxes are close based on proximity threshold
            if (abs(x - cx) < proximity_threshold and abs(y - cy) < proximity_threshold) or \
               (abs((x + w) - (cx + cw)) < proximity_threshold and abs((y + h) - (cy + ch)) < proximity_threshold):
                # Merge boxes
                nx = min(x, cx)
                ny = min(y, cy)
                nw = max(x + w, cx + cw) - nx
                nh = max(y + h, cy + ch) - ny
                combined_boxes[i] = (nx, ny, nw, nh)
                merged = True
                break
        if not merged:
            combined_boxes.append((x, y, w, h))

    return combined_boxes

In [31]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

CROP_X = 357
CROP_Y = 256
CROP_WIDTH = 574  
CROP_HEIGHT = 1002 

# Paths to images
background_image_path = './datasets/video-frames/background.png'  # Frame with just the background
background = cv2.imread(background_image_path)

def cleanup(frame, suppress_others=True, enlarge=False, kernel_size=7, iterations=3):
    # Ensure images are the same size
    if background.shape != frame.shape:
        print("Error: Background and bee frame must have the same dimensions.")
        return
    else:
        # Subtract the background directly in color
        diff = cv2.absdiff(frame, background)

        # Convert diff to HSV
        hsv_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2HSV)

        # Define HSV ranges for red, yellow, and orange
        lower_red1 = np.array([0, 100, 100])     # Lower range for red (hue 0-10)
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 100, 100])   # Upper range for red (hue 160-180)
        upper_red2 = np.array([180, 255, 255])
        lower_yellow = np.array([20, 100, 100])  # Range for yellow
        upper_yellow = np.array([30, 255, 255])
        lower_orange = np.array([10, 100, 100])  # Range for orange
        upper_orange = np.array([20, 255, 255])

        # Create masks for each color
        red_mask1 = cv2.inRange(hsv_diff, lower_red1, upper_red1)
        red_mask2 = cv2.inRange(hsv_diff, lower_red2, upper_red2)
        yellow_mask = cv2.inRange(hsv_diff, lower_yellow, upper_yellow)
        orange_mask = cv2.inRange(hsv_diff, lower_orange, upper_orange)

        # Combine all masks
        combined_mask = cv2.bitwise_or(red_mask1, red_mask2)
        combined_mask = cv2.bitwise_or(combined_mask, yellow_mask)
        combined_mask = cv2.bitwise_or(combined_mask, orange_mask)

        if enlarge:
            # Dilate the mask to enlarge the regions
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
            combined_mask = cv2.dilate(combined_mask, kernel, iterations=iterations)

        # Enhance the selected regions
        enhanced = cv2.addWeighted(diff, 1.0, diff, 0.1, 0)
        enhanced_regions = cv2.bitwise_and(enhanced, enhanced, mask=combined_mask)

        if suppress_others:
            # Suppress non-target colors
            inverse_mask = cv2.bitwise_not(combined_mask)  # Non-target areas
            suppressed = cv2.addWeighted(diff, 0.7, diff, 0.0, 0)  # Reduce intensity of non-target areas
            suppressed_regions = cv2.bitwise_and(suppressed, suppressed, mask=inverse_mask)

            # Combine enhanced and suppressed regions
            final_result = cv2.addWeighted(enhanced_regions, 1, suppressed_regions, 1.0, 0)
        else:
            final_result = enhanced_regions

        return final_result

In [ ]:
       
# Define video paths
video_dir = './datasets/mobile-videos'
videos = ["cropped_muted_1.mp4"]
from matplotlib import pyplot as plt


# Define output directory
output_dir = './datasets/pre-processed-mobile-videos'
os.makedirs(output_dir, exist_ok=True)


# Process each video
for video_name in videos:
    video_path = os.path.join(video_dir, video_name)
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open {video_name}")
        continue


    # Output video writer
    output_video_path = os.path.join(output_dir, f"boxed_{video_name}")
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 29, # 29 can be given as an return value of the pirnt_details method
                          (CROP_WIDTH, CROP_HEIGHT))

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        #-----BEE DETECTION-----
        processed_frame = cleanup(frame, enlarge=True)
        processed_frame[np.all(processed_frame < 20, axis=-1)] = 0
        non_black_mask = cv2.inRange(processed_frame, (1, 1, 1), (255, 255, 255))  # Anything above (0, 0, 0)

        # Find contours of non-black regions
        contours, _ = cv2.findContours(non_black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        rectangles = []

        # Draw bounding boxes around detected non-black regions
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 15 and h > 15:  # Exclude very small detections
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box #this is not needed    
                rectangles.append((x, y, w, h))  # Collect rectangle coordinates
        
        # Combine close rectangles
        combined_boxes = combine_drawn_boxes(rectangles, proximity_threshold=30)
        number_of_bees = len(combined_boxes)

        # Draw combined bounding boxes
        for x, y, w, h in combined_boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Draw final boxes in Blue
        

 
        #-----POLLEN DETECTION-----
        only_pollen_frame = cleanup(frame, enlarge=True, suppress_others=False)
        # Find contours of non-black regions
        non_black_mask = cv2.inRange(only_pollen_frame, (5, 5, 5), (255, 255, 255))  # Anything above (5, 5, 5) -- try out more thresholds
        

        contours, _ = cv2.findContours(non_black_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # Store all drawn boxes
        rectangles = []

        # Draw bounding boxes around detected non-black regions
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w > 20 and h > 20:  # Exclude very small detections
                # cv2.rectangle(frame, (x, y), (x + w, y + h), (40, 155, 0), 2)  # Green bounding box    
                rectangles.append((x, y, w, h))  # Collect rectangle coordinates
        
        # Combine close rectangles
        combined_boxes = combine_drawn_boxes(rectangles, proximity_threshold=50)
        number_of_pollen = len(combined_boxes)

        # Draw combined bounding boxes
        for x, y, w, h in combined_boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Draw final boxes in Red
        
        
        # Add number_of_bees and number_of_pollen text to the frame in two lines
        text1 = f"Number of Bees: {number_of_bees}"
        text2 = f"Number of Bees with Pollen: {number_of_pollen}"

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color_bee = (255, 0, 0 )  # Blue color
        color_pollen = (0, 0, 255)  # Red color
        thickness = 2

        # Calculate text size for alignment
        text1_size = cv2.getTextSize(text1, font, font_scale, thickness)[0]
        text2_size = cv2.getTextSize(text2, font, font_scale, thickness)[0]

        # Define text positions
        text_x = 10
        text1_y = text1_size[1] + 10  # Add padding for the first line
        text2_y = text1_y + text2_size[1] + 10  # Add padding for the second line

        # Add text to the frame
        cv2.putText(frame, text1, (text_x, text1_y), font, font_scale, color_bee, thickness)
        cv2.putText(frame, text2, (text_x, text2_y), font, font_scale, color_pollen, thickness)
     
        out.write(frame)

        # # Display every 1000th frame
        # if frame_idx % 200 == 0:
        #     display_frame(processed_frame, title=f"Processed Frame {frame_idx}")

        frame_idx += 1

    cap.release()
    out.release()
    print(f"Finished processing {video_name}. Saved to {output_video_path}")

    


Finished processing cropped_muted_1.mp4. Saved to ./datasets/pre-processed-mobile-videos/w_boxed_cropped_muted_1.mp4
